## Data Gathering Notebook
- Use this book to connect to the University of Minnesota Safegraph database
- The data for temporal_clminer.py exists in the monthly_patterns table
- Data is only available for the month of March

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
import psycopg2
import pandas.io.sql as sqlio

In [2]:
conn = psycopg2.connect(host="us-ppostgres02.uservices.umn.edu",
                        port="5432",database="safegraph",
                        user="lixx4266", password="rvs87bum1ya4jetey")
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print (cursor.fetchall())

sql = "select * from core_places where region = 'MN'"
core_pls = sqlio.read_sql_query(sql, conn)

print (list(core_pls.columns))

[('monthly_patterns',), ('core_places_old',), ('weekly_patterns',), ('socialdistancing_v2',), ('patterns',), ('spatial_ref_sys',), ('core_places',), ('weekly_patterns_v2a',), ('weekly_patterns_v2',)]
['placekey', 'parent_placekey', 'location_name', 'safegraph_brand_ids', 'brands', 'top_category', 'sub_category', 'naics_code', 'latitude', 'longitude', 'street_address', 'city', 'region', 'postal_code', 'iso_country_code', 'phone_number', 'open_hours', 'category_tags', 'opened_on', 'closed_on', 'tracking_closed_since', 'geometry_type']


In [3]:
open_date = core_pls[~core_pls['opened_on'].isna()][['placekey','location_name','city','brands','top_category','sub_category','opened_on','closed_on','tracking_closed_since','latitude','longitude']]

In [4]:
print(len(core_pls))
print(len(core_pls[core_pls['city'] == 'Minneapolis']))
print(len(core_pls[(core_pls['city'] == 'Minneapolis') & (core_pls['brands'] != '')]))
print(len(open_date[(open_date['city'] == 'Minneapolis') & (open_date['brands'] != '')]))

108655
8278
929
65


In [11]:
core_pls[(core_pls['city'] == 'Minneapolis') & (core_pls['brands'] != '')].to_csv('data/Minneapolis_emergent.csv',index=False)

In [22]:
grouped = open_date.groupby('city')

In [32]:
counts = grouped.size()[grouped.size() > 10].sort_values(ascending=False).reset_index(name='counts')
counts

,city,counts
0,Minneapolis,65
1,Rochester,48
2,Bloomington,32
3,Saint Paul,29
4,Roseville,23
5,Blaine,23
6,Burnsville,22
7,Duluth,21
8,Woodbury,21
9,Maple Grove,20


In [14]:
print(len(open_date), len(open_date[['placekey','opened_on']].drop_duplicates()))

1205 1205


In [15]:
print('No closed:', len(open_date[open_date['closed_on'].isna()]))

No closed: 1205


In [16]:
print(open_date['opened_on'].min(),open_date['opened_on'].max())

2019-09-24 2021-09-04


In [3]:
sql = "select * from monthly_patterns limit 20"
mpatterns = sqlio.read_sql_query(sql,conn)

In [6]:
mpatterns.drop(columns=['postal_code','street_address','visitor_country_of_origin','distance_from_home','median_dwell','bucketed_dwell_times'])

,safegraph_place_id,location_name,city,region,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,visitor_home_cbgs,visitor_work_cbgs,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,iso_country_code
0,sg:d63872bccce24e38a2c37c53fe9b9e60,Scissor Sensation,Middletown,OH,None,1580515200,1583020800,30,20,"[3, 0, 1, 0, 2, 1, 3, 1, 0, 0, 0, 0, 0, 1, 1, ...",{},{},"{'Buffalo Wild Wings': 50, 'Dollar General': 3...","{'Kroger': 48, 'Frisch's Big Boy': 45, 'Chick-...","[0, 0, 0, 0, 0, 0, 0, 1, 4, 8, 11, 6, 6, 9, 4,...","{'Monday': 3, 'Tuesday': 2, 'Wednesday': 3, 'T...","{'android': 8, 'ios': 12}",US
1,sg:d655f1c9349544a48b7c0dcf6eb980cb,Exxon Mobil,Ogden,UT,Exxon Mobil,1580515200,1583020800,118,104,"[8, 2, 1, 3, 4, 5, 3, 4, 2, 3, 3, 3, 7, 5, 4, ...","{'490572018001': 5, '490572105112': 5}",{},"{'7-Eleven': 33, 'Common Cents': 25, 'Quick La...","{'Walmart': 60, 'McDonald's': 50, 'Maverik': 4...","[1, 1, 2, 0, 1, 1, 0, 4, 8, 3, 7, 6, 13, 8, 13...","{'Monday': 11, 'Tuesday': 16, 'Wednesday': 18,...","{'android': 76, 'ios': 28}",US
2,sg:d65ad21deb214d3899614d987704e00e,Phase 2,Eastover,SC,None,1580515200,1583020800,17,7,"[1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",{},{},{'IGA': 100},"{'Dollar General': 52, 'Shell Oil': 37, 'McDon...","[3, 2, 2, 1, 1, 1, 0, 0, 1, 1, 2, 1, 3, 2, 2, ...","{'Monday': 0, 'Tuesday': 3, 'Wednesday': 4, 'T...","{'android': 0, 'ios': 0}",US
3,sg:d675f1dcaa78426eac6144cb143fb94d,NAPA Auto Parts,Aynor,SC,NAPA Auto Parts,1580515200,1583020800,234,78,"[5, 1, 9, 6, 8, 7, 6, 5, 6, 14, 10, 6, 1, 14, ...","{'450510801021': 13, '450510801012': 12, '4505...",{'450510801021': 5},"{'Speedway': 25, 'KFC': 20, 'Dollar General': ...","{'Walmart': 54, 'Dollar General': 51, 'McDonal...","[16, 22, 22, 22, 23, 23, 26, 19, 22, 18, 30, 1...","{'Monday': 53, 'Tuesday': 29, 'Wednesday': 30,...","{'android': 39, 'ios': 39}",US
4,sg:d6f5039733094ed08d06974c6fa13a74,Bamberger Nature Park,San Antonio,TX,None,1580515200,1583020800,1312,509,"[49, 54, 44, 36, 27, 41, 50, 52, 39, 30, 38, 4...","{'480291818192': 60, '480291818202': 21, '4802...","{'480291818201': 50, '480291818242': 18, '4802...","{'Twin Peaks Restaurant': 7, 'Chick-fil-A': 7,...","{'H-E-B': 38, 'Valero Energy': 38, 'Whataburge...","[258, 241, 218, 210, 200, 200, 211, 219, 226, ...","{'Monday': 146, 'Tuesday': 150, 'Wednesday': 1...","{'android': 167, 'ios': 345}",US
5,sg:d70230c9edf6487ebadd5d27aecfe396,Prosper Senatobia,Senatobia,MS,None,1580515200,1583020800,314,103,"[15, 16, 9, 12, 10, 5, 10, 7, 6, 12, 9, 10, 12...",{'281379504002': 36},{'281379504002': 23},"{'Burger King': 16, 'Dunkin'': 13, 'Popeyes Lo...","{'Walmart': 51, 'Sonic': 35, 'McDonald's': 34,...","[84, 79, 78, 77, 75, 78, 77, 70, 68, 66, 57, 6...","{'Monday': 53, 'Tuesday': 37, 'Wednesday': 44,...","{'android': 28, 'ios': 75}",US
6,sg:d736d3f0793c4f308b3888f34d7f2121,Shear Art Pet Salon,Cathedral City,CA,None,1580515200,1583020800,61,58,"[6, 1, 3, 2, 1, 1, 1, 3, 1, 2, 0, 2, 3, 5, 2, ...",{},{},{},"{'McDonald's': 43, 'Walmart': 30, 'ARCO': 29, ...","[1, 1, 2, 2, 1, 1, 1, 8, 3, 3, 8, 10, 9, 7, 4,...","{'Monday': 8, 'Tuesday': 7, 'Wednesday': 6, 'T...","{'android': 31, 'ios': 27}",US
7,sg:d85ec965971f4c2ebf063a1748923baf,DYER CHEVROLET,Vero Beach,FL,Chevrolet,1580515200,1583020800,361,203,"[8, 2, 13, 10, 10, 11, 9, 11, 8, 10, 15, 11, 1...","{'120610509031': 12, '120610508043': 9, '12061...",{'120610501007': 5},"{'Cumberland Farms': 20, 'Mazda': 11, 'Einstei...","{'Publix Super Markets': 44, 'Walmart': 31, 'M...","[2, 1, 0, 0, 0, 0, 3, 38, 99, 103, 114, 110, 1...","{'Monday': 60, 'Tuesday': 56, 'Wednesday': 60,...","{'android': 93, 'ios': 110}",US
8,sg:d7500de1d1ed43e3866fcecbe4be774b,Matthews Square Plaza,Youngstown,OH,None,1580515200,1583020800,1523,1063,"[121, 37, 31, 26, 26, 41, 58, 125, 52, 29, 21,...","{'390998113005': 24, '390998119011': 20, '3909...","{'390998120014': 20, '390998122001': 12, '3909...","{'Dollar Gener

In [8]:
sql = "select safegraph_place_id, location_name, city, region, street_address, brands, date_range_start, date_range_end, \
        raw_visit_counts, visits_by_day, median_dwell, bucketed_dwell_times, popularity_by_day \
        from monthly_patterns where region='MN' and city='Minneapolis'"
out = sqlio.read_sql_query(sql,conn)
print(len(out))

22508


In [9]:
out_with_latlon = pd.merge(out, core_pls[['location_name','street_address','top_category','sub_category','latitude','longitude']], on=['location_name','street_address'], how='left')
print(len(out_with_latlon))

22637


In [10]:
out_with_latlon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22637 entries, 0 to 22636
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   safegraph_place_id    22637 non-null  object 
 1   location_name         22637 non-null  object 
 2   city                  22637 non-null  object 
 3   region                22637 non-null  object 
 4   street_address        22637 non-null  object 
 5   brands                2384 non-null   object 
 6   date_range_start      22637 non-null  int64  
 7   date_range_end        22637 non-null  int64  
 8   raw_visit_counts      22637 non-null  int64  
 9   visits_by_day         22637 non-null  object 
 10  median_dwell          22637 non-null  float64
 11  bucketed_dwell_times  22637 non-null  object 
 12  popularity_by_day     22637 non-null  object 
 13  top_category          19031 non-null  object 
 14  sub_category          18804 non-null  object 
 15  latitude           

In [13]:
out_dropnan = out_with_latlon[out_with_latlon['latitude'].notna()]
out_dropnan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19076 entries, 0 to 22636
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   safegraph_place_id    19076 non-null  object 
 1   location_name         19076 non-null  object 
 2   city                  19076 non-null  object 
 3   region                19076 non-null  object 
 4   street_address        19076 non-null  object 
 5   brands                1721 non-null   object 
 6   date_range_start      19076 non-null  int64  
 7   date_range_end        19076 non-null  int64  
 8   raw_visit_counts      19076 non-null  int64  
 9   visits_by_day         19076 non-null  object 
 10  median_dwell          19076 non-null  float64
 11  bucketed_dwell_times  19076 non-null  object 
 12  popularity_by_day     19076 non-null  object 
 13  top_category          19031 non-null  object 
 14  sub_category          18804 non-null  object 
 15  latitude           

In [14]:
# out_dropnan.to_csv('data/Minneapolis_Visit_Input.csv',index=False)